In [1]:
import sys
sys.path.append('/p/home/jusers/kotobi2/juwels/hida_project/')

In [2]:
import torch
import torch.nn.functional as F

In [3]:
from torch_geometric.loader import DataLoader

from attribution_gnn1.QM9_SpecData import QM9_SpecData
from attribution_gnn1.split import save_split

from attribution_gnn1.SpectraGNN import SpectraGNN

In [4]:
import torch.multiprocessing as mp
from torch.utils.data.distributed import DistributedSampler
from torch.nn.parallel import DistributedDataParallel as DDP
from torch.distributed import init_process_group, destroy_process_group, barrier
import os
import os.path as osp

In [5]:
world_size = torch.cuda.device_count()
device = "cuda:0" if torch.cuda.is_available() else "cpu"
model_name = "spectragatv2_multigpu.pt"

In [6]:
world_size

4

In [7]:
spectragnn = SpectraGNN(
    gnn_name='gatv2',
    in_channels=[11, 128, 256, 512],
    out_channels=[128, 256, 512, 600],
    num_targets=100,
    num_layers=4,
    heads=None
).to(device)

# loading the saved model 
path_to_model = osp.join('./best_model', 
                         model_name)

if osp.exists(path_to_model):
    spectragnn.load_state_dict(torch.load(path_to_model))
else:
    print('model is not loaded')

model is not loaded


In [8]:
root = '/p/home/jusers/kotobi2/juwels/data_qm9/all_graph_data/qm9_spec_10k.pt'
qm9_spec = QM9_SpecData(root=root,
             raw_dir='/p/home/jusers/kotobi2/juwels/data_qm9/raw/',
             spectra=[])

In [11]:
idxs = save_split(
    path='/p/home/jusers/kotobi2/juwels/hida_project/data/split_files2/qm9_split_10k.npz',
    ndata=len(qm9_spec),
    ntrain=4000,
    nval=1000,
    ntest=0,
    save_split=True,
    shuffle=True, 
    print_nsample=True
)

In [16]:
len(qm9_spec)

10000

In [12]:
#train, val and test data
train_qm9 = [qm9_spec[i] for i in idxs['train']]
val_qm9 = [qm9_spec[i] for i in idxs['val']]
#test_qm9 = qm9_spec[idxs['test']]

IndexError: list index out of range

In [27]:
def run(rank, world_size, train_dataset):
    #os.environ['MASTER_ADDR'] = 'localhost'
    #os.environ['MASTER_PORT'] = '12355'
    #init_process_group('nccl', rank=rank, world_size=world_size)
    init_process_group( "nccl", "tcp://127.0.0.1:12345", rank, world_size)
    
    data_sampler = DistributedSampler(train_dataset,
                                      num_replicas=world_size,
                                      rank=rank)
    data_loader = DataLoader(train_dataset, batch_size=100, 
                             sampler=data_sampler, num_workers=0)
    torch.manual_seed(12345)
    model = SpectraGNN(
            gnn_name='gatv2',
            in_channels=[11, 128, 256, 512],
            out_channels=[128, 256, 512, 600],
            num_targets=100,
            num_layers=4,
            heads=None
        ).to(rank)
    model = DDP(model, device_ids=[rank])
    optimizer = torch.optim.AdamW(model.parameters(), lr=5e-4)
    loss_fn = torch.nn.L1Loss()
    
    for epoch in range(100):
        
        model.train()
        total_loss = 0
        total_graphs = 0
        
        for data in data_loader:
            data = data.to(rank)
            
            x, edge_index, batch_seg = data.x, \
            data.edge_index, data.batch
            
            optimizer.zero_grad()
            pred = model(x, edge_index, batch_seg)
            loss = loss_fn(pred.vies(-1, 1), 
                           data.spectrum.view(-1, 1))
            loss.backward()
            optimizer.step()
            total_loss += loss
            total_graphs += data.num_graphs
            
        train_loss_avg = total_loss / total_graphs
        
        barrier()
                    
        if epoch % 50 == 0:
            print(f"epoch {epoch} | average train loss = {train_loss_avg:.2f}")
            
    destroy_process_group()
            
if __name__ == '__main__':
    world_size = torch.cuda.device_count()
    print('Let\'s use', world_size, 'GPUs!')
    args = (world_size, qm9_spec)
    mp.spawn(run, args=args, nprocs=world_size, join=True)        

Let's use 4 GPUs!


OSError: [Errno 12] Cannot allocate memory